In [1]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from statsmodels.robust.scale import huber
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import re
import locale
import geopandas
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)


# df_indicadores_pob = pd.read_parquet(f'../data/processed/df_indicadores_pob.parquet')
# df_final_pob = pd.read_parquet('../data/processed/df_final_pob.parquet')
# df_final_pob_melt = pd.read_parquet('../data/processed/df_final_pob_melt.parquet')
# df_final_pob_melt_PC = pd.read_parquet('../data/processed/df_final_pob_melt_PC.parquet')
# CCAA_CO = pd.read_parquet('../data/processed/CCAA_CO.parquet')
# MUNI_CO = pd.read_parquet('../data/processed/MUNI_CO.parquet')
# PROV_CO = pd.read_parquet('../data/processed/PROV_CO.parquet')



In [12]:
df_final_pob=pd.read_parquet('../data/main_processed/df_final_pob.parquet')

df_indicadores_pob=pd.read_parquet('../data/main_processed/df_indicadores_pob.parquet')

df_indicadores_pob_pivot=pd.read_parquet('../data/main_processed/df_indicadores_pob_pivot.parquet')

df_final_pob_melt=pd.read_parquet('../data/main_processed/df_final_pob_melt.parquet')

df_final_pob_melt_PC=pd.read_parquet('../data/main_processed/df_final_pob_melt_PC.parquet')

CCAA_CO = pd.read_parquet('../data/main_processed/CCAA_CO.parquet')
MUNI_CO = pd.read_parquet('../data/main_processed/MUNI_CO.parquet')
PROV_CO = pd.read_parquet('../data/main_processed/PROV_CO.parquet')

df_final_pob_dropdown=pd.read_parquet('../data/main_processed/df_final_pob_dropdown.parquet')

df_final_pob_dropdown_c=pd.read_parquet('../data/main_processed/df_final_pob_dropdown_c.parquet')




In [13]:
list(df_indicadores_pob[['Unidades físicas de referencia' , 'Nº unidades']].columns)

['Unidades físicas de referencia', 'Nº unidades']

In [14]:
df_indicadores_pob_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162433 entries, 0 to 413864
Data columns (total 4 columns):
 #   Column                          Non-Null Count   Dtype   
---  ------                          --------------   -----   
 0   Nombre Ente Principal           162433 non-null  category
 1   Descripción                     162433 non-null  category
 2   Unidades físicas de referencia  162433 non-null  category
 3   Nº unidades                     162433 non-null  float64 
dtypes: category(3), float64(1)
memory usage: 3.3 MB


In [76]:
municipio_types='Andalucía'

In [87]:
df_final_pob_melt_PC

,CCAA,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,cohorte_pob,codigo_geo,Descripción,coste_efectivo_PC,Población 2018
0,Andalucía,Almería,Ayuntamiento,Abla,01-04-001-AA-000,1.000-2.000,4001,PC_Abastecimiento de agua potable,229.091814,1235.0
1,Andalucía,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,1.000-2.000,4002,PC_Abastecimiento de agua potable,67.154470,1150.0
2,Andalucía,Almería,Ayuntamiento,Adra,01-04-003-AA-000,20.000-50.000,4003,PC_Abastecimiento de agua potable,50.507831,24859.0
3,Andalucía,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,0-1.000,4004,PC_Abastecimiento de agua potable,80.781023,753.0
4,Andalucía,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,0-1.000,4005,PC_Abastecimiento de agua potable,253.247150,621.0
5,Andalucía,Almería,Ayuntamiento,Albox,01-04-006-AA-000,10.000-20.000,4006,PC_Abastecimiento de agua potable,0.000000,11696.0
6,Andalucía,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,0-1.000,4007,PC_Abastecimiento de agua potable,76.099570,814.0
7,Andalucía,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,0-1.000,4008,PC_Abastecimiento de agua potable,94.710739,528.0
8,Andalucía,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,0-1.000,4009,PC_Abastecimiento de agua potable,270.958444,135.0
...,...,...,...,...,...,...,...,...,...,...


In [86]:
sorted(list(df_final_pob_melt\
        .loc[(df_final_pob_melt['Nombre Ente Principal'] == municipio_types) & (df_final_pob_melt['coste_efectivo']>100)
        ,'Descripción'].unique()))

NameError: name 'municipio_types' is not defined

In [78]:
sorted(df_final_pob.loc[df_final_pob['CCAA']==CCAA_types,'Provincia'].unique().to_list())

['Almería',
 'Cádiz',
 'Córdoba',
 'Granada',
 'Huelva',
 'Jaén',
 'Málaga',
 'Sevilla']

In [85]:
sorted(df_final_pob_dropdown.loc[df_final_pob_dropdown['CCAA'] == CCAA_types ,'Nombre Ente Principal'].unique().to_list())

['Abla',
 'Abrucena',
 'Adamuz',
 'Adra',
 'Agrón',
 'Aguilar de la Frontera',
 'Alameda',
 'Alamedilla',
 'Albaida del Aljarafe',
 'Alboloduy',
 'Albolote',
 'Albondón',
 'Albox',
 'Albuñol',
 'Albuñuelas',
 'Albuñán',
 'Albánchez',
 'Alcalá de Guadaíra',
 'Alcalá de los Gazules',
 'Alcalá la Real',
 'Alcaracejos',
 'Alcaucín',
 'Alcaudete',
 'Alcolea',
 'Alcolea del Río',
 'Alcudia de Monteagud',
 'Alcóntar',
 'Aldeaquemada',
 'Alfacar',
 'Alfarnate',
 'Algaba (La)',
 'Algar',
 'Algarinejo',
 'Algarrobo',
 'Algatocín',
 'Algeciras',
 'Alhabia',
 'Alhama de Almería',
 'Alhaurín de la Torre',
 'Alhaurín el Grande',
 'Alhendín',
 'Alicún',
 'Alicún de Ortega',
 'Almargen',
 'Almegíjar',
 'Almendro (El)',
 'Almensilla',
 'Almería',
 'Almodóvar del Río',
 'Almogía',
 'Almonaster la Real',
 'Almonte',
 'Almuñécar',
 'Almáchar',
 'Almócita',
 'Alozaina',
 'Alpandeire',
 'Alpujarra de la Sierra',
 'Alquife',
 'Alsodux',
 'Andújar',
 'Antas',
 'Antequera',
 'Aracena',
 'Arahal',
 'Arboleas',


In [79]:
sorted(df_final_pob_melt.loc[df_final_pob_melt['CCAA'] == CCAA_types ,'Nombre Ente Principal'].unique().to_list())

['Abla',
 'Abrucena',
 'Adamuz',
 'Adra',
 'Agrón',
 'Aguilar de la Frontera',
 'Alameda',
 'Alamedilla',
 'Albaida del Aljarafe',
 'Alboloduy',
 'Albolote',
 'Albondón',
 'Albox',
 'Albuñol',
 'Albuñuelas',
 'Albuñán',
 'Albánchez',
 'Alcalá de Guadaíra',
 'Alcalá de los Gazules',
 'Alcalá la Real',
 'Alcaracejos',
 'Alcaucín',
 'Alcaudete',
 'Alcolea',
 'Alcolea del Río',
 'Alcudia de Monteagud',
 'Alcóntar',
 'Aldeaquemada',
 'Alfacar',
 'Alfarnate',
 'Algaba (La)',
 'Algar',
 'Algarinejo',
 'Algarrobo',
 'Algatocín',
 'Algeciras',
 'Alhabia',
 'Alhama de Almería',
 'Alhaurín de la Torre',
 'Alhaurín el Grande',
 'Alhendín',
 'Alicún',
 'Alicún de Ortega',
 'Almargen',
 'Almegíjar',
 'Almendro (El)',
 'Almensilla',
 'Almería',
 'Almodóvar del Río',
 'Almogía',
 'Almonaster la Real',
 'Almonte',
 'Almuñécar',
 'Almáchar',
 'Almócita',
 'Alozaina',
 'Alpandeire',
 'Alpujarra de la Sierra',
 'Alquife',
 'Alsodux',
 'Andújar',
 'Antas',
 'Antequera',
 'Aracena',
 'Arahal',
 'Arboleas',


In [65]:
MUNI_CO

,Provincia,Nombre Ente Principal,codigo_geo,LON,LAT
0,Almería,Abla,4001,-2.779540,37.141210
1,Almería,Abrucena,4002,-2.797467,37.132183
2,Almería,Adra,4003,-3.023303,36.748683
3,Almería,Albánchez,4004,-2.181359,37.286862
4,Almería,Alboloduy,4005,-2.621686,37.033396
5,Almería,Albox,4006,-2.147991,37.388508
6,Almería,Alcolea,4007,-2.960666,36.973620
7,Almería,Alcóntar,4008,-2.597569,37.336118
8,Almería,Alcudia de Monteagud,4009,-2.266790,37.235872
...,...,...,...,...,...


In [66]:
PROV_CO

,Provincia,Nombre Ente Principal,codigo_geo,LON,LAT
12,Almería,Almería,4013,-2.464132,36.838924
112,Cádiz,Espera,11017,-5.805975,36.871518
152,Córdoba,Córdoba,14021,-4.780325,37.879542
261,Granada,Granada,18087,-3.600019,37.176419
377,Huelva,Huelva,21041,-6.950406,37.260041
444,Jaén,Jaén,23050,-3.790359,37.765191
548,Málaga,Málaga,29067,-4.419975,36.720343
645,Sevilla,Sevilla,41091,-5.992514,37.386205
725,Huesca,Huesca,22125,-0.408423,42.140627
...,...,...,...,...,...


In [63]:
CCAA_CO

,Geo Point,Geo Shape,Codigo,Comunidade Autónoma,Texto,LAT,LON,CCAA
0,"35.2908279949,-2.95053552337","{""type"": ""Polygon"", ""coordinates"": [[[-2.95253...",19,Melilla,Melilla,35.290828,-2.950536,Melilla
1,"43.1975195366,-4.03001213183","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-3...",6,Cantabria,Cantabria,43.197520,-4.030012,Cantabria
2,"42.2748733608,-2.51703983986","{""type"": ""Polygon"", ""coordinates"": [[[-2.99722...",17,La Rioja,La Rioja,42.274873,-2.517040,La Rioja
3,"37.463274379,-4.5756251361","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-5...",1,Andalucía,Andalucía,37.463274,-4.575625,Andalucía
4,"42.6672011468,-1.6461117688","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-1...",15,Nafarroa,Comunidad Foral de Navarra,42.667201,-1.646112,Nafarroa
5,"43.0433630599,-2.61681792149","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-1...",16,Euskadi,País Vasco,43.043363,-2.616818,Euskadi
6,"43.292357861,-5.99350932547","{""type"": ""Polygon"", ""coordinates"": [[[-5.83988...",3,Asturias,Principado de Asturias,43.292358,-5.993509,Principado de Asturias
7,"41.7985537834,1.52905348544","{""type"": ""MultiPolygon"", ""coordinates"": [[[[0....",9,Catalunya,Cataluña,41.798554,1.529053,Cataluña
8,"39.5751889864,2.91229172079","{""type"": ""MultiPolygon"", ""coordinates"": [[[[4....",4,Illes Balears,Islas Baleares,39.575189,2.912292,Illes Balears
...,...,...,...,...,...,...,...,...


In [67]:
df_indicadores_pob_pivot

,Nombre Ente Principal,Descripción,Unidades físicas de referencia,Nº unidades
0,Abades,Abastecimiento de agua potable,Longitud de la red: metros lineales,9136.0
1,Abades,Abastecimiento de agua potable,Nº viviendas conectadas y no conectadas,543.0
2,Abades,"Abastos, mercados, lonjas",Nº puestos,0.0
3,Abades,"Abastos, mercados, lonjas",Superficie total en metros cuadrados,0.0
4,Abades,Acceso a los núcleos de población,Puntos kilométricos: P.K correspondientes al i...,1.0
5,Abades,Actividades funerarias,Nº efectivos en plantilla asignados al servicio,0.1
6,Abades,Actividades funerarias,Nº vehículos adscritos al servicio,0.1
7,Abades,Alcantarillado,Longitud del tramo: metros lineales.,11360.0
8,Abades,Alcantarillado,Nº viviendas con servicio,543.0
...,...,...,...,...


In [68]:
print(df_indicadores_pob.nunique())

df_indicadores_pob.info()

Provincia                            46
Código Ente Principal              4869
Tipo Ente Principal                   1
Nombre Ente Principal              4869
Descripción                          43
Unidades físicas de referencia       59
Nº unidades                       34509
codigoM                            4869
Población 2018                     2867
CCAA                                 15
codigo_geo                         4869
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 509703 entries, 0 to 509702
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype   
---  ------                          --------------   -----   
 0   Provincia                       509703 non-null  category
 1   Código Ente Principal           509703 non-null  category
 2   Tipo Ente Principal             509703 non-null  category
 3   Nombre Ente Principal           509703 non-null  category
 4   Descripción                     509703 non-null  cate

In [69]:
print(df_final_pob.nunique())

df_final_pob.info()

CCAA                                                                         15
Provincia                                                                    46
Tipo Ente Principal                                                           1
Nombre Ente Principal                                                      4869
Código Ente Principal                                                      4869
Abastecimiento de agua potable                                             3744
Abastos, mercados, lonjas                                                   623
Acceso a los núcleos de población                                          2102
Actividades funerarias                                                     1860
Alcantarillado                                                             2970
Alumbrado público                                                          4696
Biblioteca pública                                                         2177
                                        

In [70]:
df_final_pob['TOTAL'].sum()

28641291941.309998

In [71]:
print(df_final_pob['Población 2018'].sum())
df_indicadores_pob['Población 2018'].sum()

39376812.0


19359679860.0

In [72]:
df_final_pob_melt

,CCAA,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,cohorte_pob,codigo_geo,Descripción,coste_efectivo
0,Andalucía,Almería,Ayuntamiento,Abla,01-04-001-AA-000,1.000-2.000,4001,Abastecimiento de agua potable,282928.39
1,Andalucía,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,1.000-2.000,4002,Abastecimiento de agua potable,77227.64
2,Andalucía,Almería,Ayuntamiento,Adra,01-04-003-AA-000,20.000-50.000,4003,Abastecimiento de agua potable,1255574.16
3,Andalucía,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,0-1.000,4004,Abastecimiento de agua potable,60828.11
4,Andalucía,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,0-1.000,4005,Abastecimiento de agua potable,157266.48
5,Andalucía,Almería,Ayuntamiento,Albox,01-04-006-AA-000,10.000-20.000,4006,Abastecimiento de agua potable,0.00
6,Andalucía,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,0-1.000,4007,Abastecimiento de agua potable,61945.05
7,Andalucía,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,0-1.000,4008,Abastecimiento de agua potable,50007.27
8,Andalucía,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,0-1.000,4009,Abastecimiento de agua potable,36579.39
...,...,...,...,...,...,...,...,...,...


In [56]:
df_final_pob_melt_PC

,CCAA,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,cohorte_pob,Descripción,coste_efectivo_PC,Población 2018
0,Andalucía,Almería,Ayuntamiento,Abla,01-04-001-AA-000,1.000-2.000,PC_Abastecimiento de agua potable,229.091814,1235.0
1,Andalucía,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,1.000-2.000,PC_Abastecimiento de agua potable,67.154470,1150.0
2,Andalucía,Almería,Ayuntamiento,Adra,01-04-003-AA-000,20.000-50.000,PC_Abastecimiento de agua potable,50.507831,24859.0
3,Andalucía,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,0-1.000,PC_Abastecimiento de agua potable,80.781023,753.0
4,Andalucía,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,0-1.000,PC_Abastecimiento de agua potable,253.247150,621.0
5,Andalucía,Almería,Ayuntamiento,Albox,01-04-006-AA-000,10.000-20.000,PC_Abastecimiento de agua potable,0.000000,11696.0
6,Andalucía,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,0-1.000,PC_Abastecimiento de agua potable,76.099570,814.0
7,Andalucía,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,0-1.000,PC_Abastecimiento de agua potable,94.710739,528.0
8,Andalucía,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,0-1.000,PC_Abastecimiento de agua potable,270.958444,135.0
...,...,...,...,...,...,...,...,...,...
